In [ ]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
from sklearn.ensemble import RandomForestClassifier
import odc.geo.xr
import rioxarray
import matplotlib.pyplot as plt
import rasterio.features
from dea_tools.dask import create_local_dask_cluster

In [ ]:
import os
os.environ['REQUESTS_CA_BUNDLE'] = ''

In [ ]:
# Create local dask cluster to improve data load time
dask_client = create_local_dask_cluster(return_client=True)

In [ ]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [ ]:
year="2020"
aoi = gpd.read_file("aoi_test_pg.geojson")
bbox = rasterio.features.bounds(aoi)

In [ ]:
#aoi.explore()

In [ ]:

items = list(client.search(collections=["dep_s2_geomad"], datetime=year, bbox=bbox).items())

In [ ]:
data = load(
        items,
        measurements=[
            "nir", "red", "blue", "green",  "green",  "swir16", 
        ],
        bbox=bbox,
        resolution=10,
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day",
    )

In [ ]:
#scale
data = (data.where(data != 0) * 0.0001).clip(0, 1)

In [ ]:
data

In [ ]:
#from dea_tools.plotting import rgb
#rgb(data, bands=['red', 'green', 'blue'])

### AMMI

Automatic Mangrove Map and Index (AMMI)
Suyarso (2022) developed a mangrove vegetation index that
simultaneously extracts mangroves and computes canopy density
precisely using optical satellite imagery, e.g., Sentinel-2 and
Landsat-5, Landsat-7, and Landsat-8. The algorithm is the
product of two equations. The first equation should separate the
land and vegetation from water features. The second equation
should map the extent of mangroves and display the canopy
density. The proponent did not provide a threshold range but
from his results, it was between 5 to 10.

In [ ]:
nir = data['nir']
swir = data['swir16']
red = data['red']
#
green = data['green']
blue = data['blue']

# Convert to float32 to avoid integer division
nir = nir.astype("float32")
red = red.astype("float32")
swir1 = swir.astype("float32")
#

#ammi = ((nir - red) / (red + swir)) * ((nir - swir) / (swir - 0.65 * red))

# Prevent divide-by-zero by adding a tiny epsilon where denominators are zero
eps = 1e-8
denom1 = (red + swir1).where((red + swir1) != 0, eps)
denom2 = (swir1 - 0.65 * red).where((swir1 - 0.65 * red) != 0, eps)
ammi = ((nir - red) / denom1) * ((nir - swir1) / denom2)

ammi = ammi.to_dataset(name='ammi')
ammi

In [ ]:
#Spatially, the AMMI captures the mangroves from sparse mangroves, 
#indicated by the low spectral sensitivity with an index of about 5, to dense mangroves (>20), 
#and the index below 5 is classified as nonmangrove
#https://onlinelibrary.wiley.com/doi/10.1155/2022/8103242

#AMMI_THRESHOLD = 4.0 - 20
AMMI_THRESHOLD = range(4, 20)

mangrove_mask = (ammi.ammi >= list(AMMI_THRESHOLD)[0])

for i, val in enumerate(AMMI_THRESHOLD, 1):
	#print(f"{i}: {val}")
    mangrove_mask = xr.where(ammi.ammi >= val, i, mangrove_mask)


# Convert boolean mask to uint8 (0/1) and attach geospatial metadata for saving
mangrove_mask = mangrove_mask.astype("uint8")
mangrove_mask = mangrove_mask.compute()

#remove null
mangrove_mask_sanitised = mangrove_mask.where(mangrove_mask != 0, drop=True)


In [ ]:
#morphological clean up

In [ ]:
mangrove_mask_sanitised.plot()
#ammi.ammi.plot()

### Enhanced Mangrove Index (EMI)

In [ ]:
#Enhanced Mangrove Index (EMI)
#emi = (nir - swir) / (green + nir)
#emi = emi.to_dataset(name='emi')
#emi

In [ ]:
#emi.emi.plot()
mangrove_mask_sanitised.spatial_ref

### Visual Verification

In [ ]:
gdf = gpd.read_parquet("gmw_v4_pacific_validation.parquet")
aoi = aoi.to_crs(3832)
gdf = gdf.clip(aoi, keep_geom_type=False)
gdf.COUNTRY.unique()

In [ ]:
import folium
#m = folium.Map(location=[float(aoi.centroid.y[0]), float(aoi.centroid.x[0])], zoom_start=12)
aoi_1 = aoi.to_crs(4326)
center_lat = aoi_1.centroid.y.mean()
center_lon = aoi_1.centroid.x.mean()

# Initialize the Folium map centered on the average centroid
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

In [ ]:

#gmw4-validation
gdf.explore(m=m, name="GMWv4")

#rgb
rgb_ds = data.odc.to_rgba(vmin=0, vmax=0.3, bands=("red", "green", "blue"))
rgb_ds.odc.add_to(m, name="GeoMAD True Color")

#mangrove
mangrove_mask_sanitised.rio.write_crs("EPSG:3832", inplace=True)
mangrove_mask_1 = mangrove_mask_sanitised.rio.reproject("EPSG:4326")
mangrove_mask_sanitised.odc.add_to(m, cmap="viridis", name="AMMI")

folium.LayerControl().add_to(m)
m


### Export

In [ ]:
odc.geo.xr.write_cog(mangrove_mask_1, f"output_{year}.tif", overwrite=True)